# Bilgisayar Fiyat Tahmini – Regresyon Modeli (Linear Regression)

**Ad:** Yusuf

**Soyad:** TUNÇ

**Okul No:** 2012721024

**GitHub Repo Bağlantısı:** https://github.com/ytunc4141/MLP_Flask

Bu projede, bilgisayar donanım özellikleri (RAM, SSD, İşlemci, Ekran Kartı vb.) kullanılarak **Linear Regression** algoritması ile fiyat tahmini yapılması amaçlanmıştır.

Proje şu adımları içermektedir:
* Veri temizleme ve düzenleme.
* Özellik mühendisliği (Metin tabanlı işlemci/ekran kartı verilerinin puanlanması).
* Geriye doğru eleme (Backward Elimination) ile istatistiksel analiz.
* Modelin eğitilmesi ve hata oranlarının (MAE, R2) hesaplanması.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import pickle
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## 1. Veri Setinin Yüklenmesi ve Düzenlenmesi
Veri seti Excel formatından okunur. RAM ve SSD sütunlarındaki "GB" gibi ifadeler temizlenerek sayısal formata çevrilir.

In [2]:
df = pd.read_excel('/content/drive/MyDrive/Colab_Notebooks/MLP_Flask/dataset.xlsx')

df['RAM_GB'] = pd.to_numeric(df['RAM_GB'], errors='coerce')
df['SSD_GB'] = pd.to_numeric(df['SSD_GB'], errors='coerce')

df['RAM_GB'] = df['RAM_GB'].fillna(df['RAM_GB'].mean())
df['SSD_GB'] = df['SSD_GB'].fillna(df['SSD_GB'].mean())

print("Veri seti başarıyla yüklendi ve temizlendi.")
display(df.head(3))

Veri seti başarıyla yüklendi ve temizlendi.


,Marka,Islemci_Model,RAM_GB,SSD_GB,Ekran_Karti,VRAM_GB,Kasa_RGB,Garanti_Yil,Fiyat
0,AMD,Ryzen 5 7500F,16,1024.0,RTX 4060,8,1,2,45000
1,Intel,i5-14600KF,32,1024.0,RTX 4060,8,1,2,55000
2,Intel,i9-14900K,32,1024.0,RTX 4070,12,1,2,100000


## 2. Özellik Mühendisliği (Feature Engineering)
Bu aşamada, modelin metin verilerini (İşlemci ismi, Ekran kartı modeli) anlayabilmesi için özel puanlama fonksiyonları kullanıyoruz.
* **İşlemci Puanlama:** i9/Ryzen 9 -> 4 Puan, i7 -> 3 Puan vb.
* **GPU Puanlama:** RTX 4090 -> 5 Puan, RTX 3060 -> 2 Puan vb.

In [3]:
def puanla_islemci(text):
    text = str(text).lower()
    if '19' in text or 'ryzen 9' in text: return 4
    elif 'i7' in text or 'ryzen 7' in text: return 3
    elif 'i5' in text or 'ryzen 5' in text: return 2
    elif 'i3' in text or 'ryzen 3' in text: return 1
    else: return 2

def puanla_gpu(text):
    text = str(text).lower()
    if any(x in text for x in ['4090', '4080', '7900']): return 5
    elif any(x in text for x in ['4070', '3080', '6800', '7800']): return 4
    elif any(x in text for x in ['4060', '3070', '6700', '3060 ti']): return 3
    elif any(x in text for x in ['3060', '3050', '6600', '2060']): return 2
    else: return 1

df['Marka_Puan'] = df['Marka'].apply(lambda x: 1 if 'Intel' in str(x) else 0)
df['Islemci_Puan'] = df['Islemci_Model'].apply(puanla_islemci)
df['GPU_Puan'] = df['Ekran_Karti'].apply(puanla_gpu)

print("Nitel veriler nicel verilere (puanlara) dönüştürüldü.")

Nitel veriler nicel verilere (puanlara) dönüştürüldü.


## 3. İstatistiksel Analiz (Backward Elimination)
Hangi özelliklerin fiyat üzerinde gerçekten etkisi olduğunu anlamak için OLS (Ordinary Least Squares) analizi yapıyoruz. P-değeri yüksek olan (anlamsız) sütunlar modelden çıkarılır.

In [4]:
X_analiz = df[['Marka_Puan', 'Islemci_Puan', 'RAM_GB', 'SSD_GB', 'GPU_Puan', 'VRAM_GB', 'Kasa_RGB', 'Garanti_Yil']]
y = df['Fiyat']

X_opt = sm.add_constant(X_analiz)
model_stats = sm.OLS(y, X_opt).fit()

print("--- BACKWARD ELIMINATION ÖNCESİ ANALİZ ---")
print(model_stats.summary())

print("\nAnaliz sonucu: 'Kasa_RGB' ve 'Garanti_Yil' değişkenleri etkisi düşük olduğu için elendi.")

X_final = df[['Marka_Puan', 'Islemci_Puan', 'RAM_GB', 'SSD_GB', 'GPU_Puan', 'VRAM_GB']]

--- BACKWARD ELIMINATION ÖNCESİ ANALİZ ---
                            OLS Regression Results                            
Dep. Variable:                  Fiyat   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     45.78
Date:                Wed, 24 Dec 2025   Prob (F-statistic):           5.08e-18
Time:                        08:42:01   Log-Likelihood:                -565.80
No. Observations:                  50   AIC:                             1150.
Df Residuals:                      41   BIC:                             1167.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const

## 4. Model Eğitimi ve Test
Veri seti %80 Eğitim, %20 Test olacak şekilde ayrılır ve Linear Regression modeli eğitilir.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

lm = LinearRegression()
lm.fit(X_train, y_train)

y_pred = lm.predict(X_test)

print("Model başarıyla eğitildi.")

Model başarıyla eğitildi.


## 5. Sonuçların Değerlendirilmesi ve Kayıt
Modelin başarısını ölçmek için $R^2$ (Belirlilik Katsayısı) ve MAE (Ortalama Mutlak Hata) metriklerine bakılır. Son olarak model, web uygulamasında kullanılmak üzere `model.pkl` olarak kaydedilir.

In [6]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Model Başarısı (R2): {r2:.2f}")
print(f"Ortalama Hata (MAE): {mae:.0f} TL")

pickle.dump(lm, open('model.pkl', 'wb'))
print("Model başarıyla 'model.pkl' dosyasına kaydedildi! Flask uygulamasına geçebilirsiniz.")

Model Başarısı (R2): 0.83
Ortalama Hata (MAE): 23713 TL
Model başarıyla 'model.pkl' dosyasına kaydedildi! Flask uygulamasına geçebilirsiniz.
